Instructions:
1. Connect to a runtime with gpu (likely T4 because it's free)
2. Drag the Clean Georgia Data.csv into the file bar on the left
3. Go to runtime in the top left and press run all


Be warned: It's not always 100% accurate

# Setup 5-8 mins

In [7]:
!pip install unsloth
!pip install peft
!pip install gradio

from unsloth import FastLanguageModel
from peft import PeftModel

# Load the base model with FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)

==((====))==  Unsloth 2025.1.6: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu118. CUDA: 7.5. CUDA Toolkit: 11.8. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [8]:
!pip install scikit-learn
!pip install pandas
!pip install nltk
!pip install sentence-transformers

In [22]:
base_model_name = "unsloth/Llama-3.2-3B-Instruct"
adapter_path = "jaspersands/model"  # Path to your LoRA adapter on Hugging Face

model = PeftModel.from_pretrained(model, adapter_path)

/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0

In [41]:
#newwww
import pandas as pd
from unsloth.chat_templates import get_chat_template
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
import nltk

# Ensure you have NLTK stopwords downloaded
nltk.download("stopwords")
from nltk.corpus import stopwords

# Step 1: Load the CSV file
file_path = '/content/data/Clean Missouri Data.csv'
df = pd.read_csv(file_path, encoding='MacRoman')

# Step 2: Define a function to search relevant policies based on the user's query using cosine similarity
def search_relevant_policies(query, df, top_n=10):
    # Convert policies into a TF-IDF matrix
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['Content'])

    # Get the query as a TF-IDF vector
    query_vector = tfidf.transform([query])

    # Calculate cosine similarity between query and policies
    cosine_sim = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Get the top N relevant policies
    top_indices = cosine_sim.argsort()[-top_n:][::-1]
    relevant_policies = df.iloc[top_indices]

    return relevant_policies

def get_content_after_query(response_text, query):
    # Find the position of the query within the response text
    query_position = response_text.lower().find(query.lower())
    if query_position != -1:
        # Return the content after the query position
        return response_text[query_position + len(query):].strip()
    else:
        # If the query is not found, return the full response text as a fallback
        return response_text.strip()


def process_query(query,tokenizer):

    relevant_policies = search_relevant_policies(user_query, df)

    # Step 5: Combine the relevant policies with the user's query for the model
    formatted_policies = []
    for index, row in relevant_policies.iterrows():
        formatted_policy = f"Title: {row['Title']}\nTerritory: {row['Territory']}\nType: {row['Type']}\nYear: {row['Year']}\nCategory: {row['Category']}\nFrom: {row['From']}\nTo: {row['To']}\nContent: {row['Content']}\nLink: {row['Link to Content']}\n"
        formatted_policies.append(formatted_policy)
    relevant_policy_text = "\n\n".join(formatted_policies)

    # Messages with relevant policies for the model
    messages_with_relevant_policies = [
        {"role": "system", "content": relevant_policy_text},
        {"role": "user", "content": user_query},
    ]

    # Step 6: Apply chat template and tokenize
    tokenizer = get_chat_template(
        tokenizer,
        chat_template="llama-3.1",
    )
    inputs = tokenizer.apply_chat_template(
        messages_with_relevant_policies,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    FastLanguageModel.for_inference(model)
    outputs = model.generate(input_ids=inputs, max_new_tokens=256, use_cache=True, temperature=1.5, min_p=0.1)

    # Step 7: Decode the output
    generated_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    response = get_content_after_query(generated_response, user_query)

    # Step 8: Rank the top 10 policies using SBERT for the final link
    # Load SBERT model
    model_sbert = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose another SBERT model if desired

    # Encode the generated response using SBERT
    response_embedding = model_sbert.encode(generated_response, convert_to_tensor=True)

    # Encode each policy in the top 10 list
    policy_embeddings = model_sbert.encode(relevant_policies['Content'].tolist(), convert_to_tensor=True)

    # Calculate cosine similarities between the generated response and each policy embedding
    cosine_similarities = util.cos_sim(response_embedding, policy_embeddings).flatten()

    # Identify the policy with the highest SBERT cosine similarity score
    most_relevant_index = cosine_similarities.argmax().item()
    most_relevant_link = relevant_policies.iloc[most_relevant_index]['Link to Content']

    # Print the link to the most relevant source
    return {
        "response": response,
        "most_relevant_link": most_relevant_link
    }

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Final run 1min

In [44]:
user_query = "How does one qualify to be a foster parent"
result = process_query(user_query, tokenizer)
print("Query: ", user_query)
print("Response:", result["response"])
print("Most Relevant Link:", result["most_relevant_link"])

Query:  How does one qualify to be a foster parent
Response: assistant

To qualify to be a foster parent, an individual or couple must meet the eligibility requirements established by the state or local child welfare agency. These requirements typically include:

1. **Age and Health**: Foster parents must be at least 21 years old and of sound physical and mental health. Certain medical conditions may make an individual ineligible to become a foster parent.
2. **Licensing**: Foster parents must obtain a foster home license from the state or local child welfare agency. This requires completing a training program, passing a background check, and providing proof of insurance.
3. **Training**: Foster parents must complete a training program approved by the state or local child welfare agency. This training covers topics such as child development, behavior management, and crisis intervention.
4. **Background Check**: Foster parents must undergo a thorough background check, including a finger

You can just run this cell and change the query if the setup has already run once

You'll see an original which should have a link at the end that maybe or may not lead to a real database page as well as a second link which is a direct copy from the database.

In [45]:
import gradio as gr

def gradio_process_query(user_query):
    """
    This function calls your existing 'process_query' logic
    to generate a response and fetch the most relevant link.
    """
    return(process_query(user_query, tokenizer))

In [46]:
# Define the Gradio interface
iface = gr.Interface(
    fn=gradio_process_query,
    inputs=gr.Textbox(label="Enter Your Query"),
    outputs=[
        gr.Textbox(label="Response"),
        gr.Textbox(label="Most Relevant Link")
    ],
    title="Georgia Policy Chatbot",
    description="Ask a question about Georgia's policy data."
)

# Launch the interface
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://069826dcabede54259.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
